In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer



# Fungsi untuk membersihkan teks
def text_clean(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    sastrawi = StopWordRemoverFactory()
    stopworda = sastrawi.get_stop_words()
    clean_spcl = re.compile('[/(){}\[\]\|@,;]')
    clean_symbol = re.compile('[^0-9a-z #+_]')
    text = text.lower()
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    text = stemmer.stem(text)
    text = ' '.join(word for word in text.split() if word not in stopworda)
    return text

# Fungsi untuk menampilkan data wisata
def display_data_wisata():
    data_wisata = pd.read_excel('data_wisata_madura.xlsx', sheet_name='Data Wisata Madura1', usecols='A:E')
    data_wisata = data_wisata[data_wisata['description'].notnull()]
    data_wisata.reset_index(drop=True, inplace=True)
    print(data_wisata[['place_name', 'description']])

# Fungsi untuk menampilkan grafik wisata di Madura
def display_grafik_wisata():
    data_wisata = pd.read_excel('data_wisata_madura.xlsx', sheet_name='Data Wisata Madura1', usecols='A:E')
    data_wisata = data_wisata[data_wisata['description'].notnull()]
    
    # Pie chart: Total Wisata per Kabupaten
    wisata_per_kabupaten = data_wisata['city'].value_counts()
    kabupaten = wisata_per_kabupaten.index
    jumlah_wisata = wisata_per_kabupaten.values
    colors = ['lightblue', 'lightgreen', 'orange', 'yellow']
    plt.figure(figsize=(8, 6))
    plt.pie(jumlah_wisata, labels=kabupaten, colors=colors, autopct='%1.1f%%')
    plt.title('Total Wisata per Kabupaten')
    total_wisata = sum(jumlah_wisata)
    plt.legend(title='Total Wisata: {}'.format(total_wisata), loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()

    # Bar chart: Total Jumlah Wisata per Kabupaten
    wisata_per_kabupaten = data_wisata['city'].value_counts()
    kabupaten = wisata_per_kabupaten.index
    jumlah_wisata = wisata_per_kabupaten.values
    colors = ['lightblue', 'lightgreen', 'orange', 'yellow']
    plt.figure(figsize=(10, 6))
    plt.bar(kabupaten, jumlah_wisata, color=colors)
    for i, v in enumerate(jumlah_wisata):
        plt.text(i, v, str(v), ha='center', va='bottom', fontweight='bold')
    plt.xlabel('Kabupaten')
    plt.ylabel('Jumlah Wisata')
    plt.title('Total Jumlah Wisata per Kabupaten')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # Bar chart: Jumlah Wisata per Kategori
    count_kategori = data_wisata['category'].value_counts()
    fig, ax = plt.subplots()
    ax.bar(count_kategori.index, count_kategori.values)
    for i, count in enumerate(count_kategori.values):
        ax.text(i, count, str(count), ha='center', va='bottom')
    ax.set_xlabel('Kategori Wisata')
    ax.set_ylabel('Jumlah')
    ax.set_title('Jumlah Wisata per Kategori')
    plt.tight_layout()
    plt.show()

    # Grouped bar chart: Total Jumlah Kategori Wisata per Kabupaten
    kategori_per_kabupaten = data_wisata.groupby('city')['category'].value_counts().unstack().fillna(0)
    fig, ax = plt.subplots(figsize=(10, 6))
    kategori_per_kabupaten.plot(kind='bar', ax=ax)
    for container in ax.containers:
        ax.bar_label(container)
    ax.set_xlabel('Kabupaten')
    ax.set_ylabel('Count Wisata')
    ax.set_title('Total jumlah Kategori Wisata per Kabupaten')
    plt.show()

# Fungsi untuk melakukan rekomendasi wisata
def recommend_wisata():
    data_wisata = pd.read_excel('data_wisata_madura.xlsx', sheet_name='Data Wisata Madura1', usecols='A:E')
    data_wisata = data_wisata[data_wisata['description'].notnull()]
    
    # Membersihkan data dan membuat matriks TF-IDF
    data_wisata['cleaned_description'] = data_wisata['description'].apply(text_clean)
    data_wisata.set_index('place_name', inplace=True)
    tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0,)
    tfidf_matrix = tf.fit_transform(data_wisata['cleaned_description'])
    cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    indices = pd.Series(data_wisata.index)

    def recommendations(keyword, top=10):
        rekomendasi_judul = []
        rekomendasi_deskripsi = []
        cleaned_keyword = text_clean(keyword) 
        tfidf_keyword = tf.transform([cleaned_keyword])
        cos_sim_keyword = cosine_similarity(tfidf_matrix, tfidf_keyword)

        # Cari berdasarkan judul
        idx = indices[indices == keyword].index
        if len(idx) > 0:
            idx = idx[0]
            score_series = pd.Series(cos_sim_keyword.flatten(), index=indices).sort_values(ascending=False)
            top_indexes = list(score_series.iloc[:top].index)

            for i in top_indexes:
                place_name = i
                kabupaten = data_wisata['city'][place_name]
                description = data_wisata['cleaned_description'][place_name]
                score = score_series[i]
                if place_name.lower() == cleaned_keyword.lower():  # mengecek keyword yang dimasukan yang specific dengan tanpa lower
                    score = 1.0  # merubah scire jika inputan keyword cocok dengan placename
                rekomendasi_judul.append((place_name, kabupaten, description, score))

        # Cari berdasarkan deskripsi jika tidak ada rekomendasi berdasarkan judul atau tidak ada input yang sesuai dengan judul
        if len(rekomendasi_judul) == 0 or cleaned_keyword.lower() not in [name.lower() for name in data_wisata.index]:
            cleaned_keyword_lower = cleaned_keyword.lower() 
            tfidf_keyword_lower = tf.transform([cleaned_keyword_lower])
            scores_deskripsi = cosine_similarity(tfidf_matrix, tfidf_keyword_lower).flatten()
            sorted_indexes_deskripsi = np.argsort(scores_deskripsi)[::-1]

            for i in sorted_indexes_deskripsi:
                place_name = list(data_wisata.index)[i]
                kabupaten = data_wisata['city'][place_name]
                description = data_wisata['cleaned_description'][place_name]
                score = scores_deskripsi[i]
                if score >= 0.01:
                    rekomendasi_deskripsi.append((place_name, kabupaten, description, score))

        # Gabungkan rekomendasi judul dan deskripsi
        rekomendasi_wisata = rekomendasi_judul + rekomendasi_deskripsi
        # Urutkan berdasarkan skor
        rekomendasi_wisata.sort(key=lambda x: x[3], reverse=True)
        # Ambil jumlah rekomendasi sesuai dengan top
        rekomendasi_wisata = rekomendasi_wisata[:top]
        return rekomendasi_wisata

    keyword = input("Masukkan keyword: ")
    jumlah_rekomendasi = int(input("Masukkan jumlah rekomendasi: "))
    hasil_rekomendasi = recommendations(keyword, jumlah_rekomendasi)
    print("Hasil rekomendasi wisata yang mungkin Anda sukai : ",keyword)
    for index, rekomendasi in enumerate(hasil_rekomendasi, 1):
        place_name, kabupaten, description, score = rekomendasi
        print("Rekomendasi ke-{}".format(index))
        print("Nama Wisata:", place_name)
        print("Kabupaten:", kabupaten)
        print("Deskripsi Bersih:", description)
        print("Score:", score)
        print()


# Main menu
while True:
    print("========== MENU ==========")
    print("1. Data Wisata")
    print("2. Grafik Wisata di Madura")
    print("3. Rekomendasi Wisata")
    print("0. Keluar")
    choice = input("Pilih menu: ")

    if choice == "1":
        print("\n----- Data Wisata -----")
        display_data_wisata()
        print("-----------------------\n")
    elif choice == "2":
        print("\n----- Grafik Wisata di Madura -----")
        display_grafik_wisata()
        print("-----------------------------------\n")
    elif choice == "3":
        print("\n----- Rekomendasi Wisata -----")
        recommend_wisata()
        print("-----------------------------\n")
    elif choice == "0":
        print("Terima kasih! Program telah berakhir.")
        break
    else:
        print("Pilihan tidak valid. Silakan pilih menu yang tersedia")


========== MENU ==========
1. Data Wisata
2. Grafik Wisata di Madura
3. Rekomendasi Wisata
0. Keluar

----- Data Wisata -----
                            place_name  \
0                      Api Alam Konang   
1                         Gunung Geger   
2    Stadion Kerapan Sapi R.P Moh Noer   
3                     Kolla Langgundih   
4                   Makam Air Mata Ibu   
..                                 ...   
145        Wisata Keris Aing Tong Tong   
146            Pantai Mamburit Arjasa    
147               Pantai Ruba Ambunten   
148         Taman Adipura Kota Sumenep   
149      Pantai Toraja / Berung Toraja   

                                           description  
0    merupakan api tak kunjung padam yang bersumber...  
1    adalah salah satu tempat wisata .yang berada d...  
2       merupakan lapangan sapi terbesar di Bangkalan   
3    merupakan tempat wisata religi yang dulunya di...  
4    adalah destinasi wisata religi di Madura yang ...  
..                         